In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
# !pip install ninja
# # Set TORCH_CUDA_ARCH_LIST if running and building on different GPU types
# !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/Lora_Mistral_Nemo", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference



==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2024.9 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [5]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Assuming you already have the test_df loaded
# Drop the image_path column
test_df = pd.read_csv('/content/testcombine_centimetre.csv')
test_df = test_df.drop(columns=['image_path'])

# Split the data into X and y
X = test_df

In [7]:
X = X.iloc[0:5000]

In [8]:
X

,index,group_id,entity_name,extracted_text
0,0,156839,height,2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in
1,1,792578,width,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
2,2,792578,height,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
3,3,792578,depth,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
4,4,792578,depth,"Size Width Length One Size 10.50cm/4.13"" 90cm/..."
...,...,...,...,...
4995,13384,635528,voltage,iPhone13 HuaWei P40 XiaoMi11 2.2Times 1.9Times...
4996,13387,416664,depth,13 in/33 cm 3.3 in/8.5cm
4997,13390,846116,width,100cm 19.5cm
4998,13391,365637,item_weight,Walkers Pure Butter SHORTBREAD tWe 150g e5.3ozD


In [ ]:
from tqdm import tqdm
# Initialize a list to store the generated responses
generated_responses = []

# Loop through each row in X
for idx, row in tqdm(X.iterrows()):
    entity_name = row['entity_name']
    extracted_text = row['extracted_text']

    # Create the prompt
    prompt_text = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Allowed Inputs:

For the following entities, only the specified units are allowed. Please ensure that the input matches one of the permitted units for each entity:

Width: centimetre, foot, inch, metre, millimetre, yard
Depth: centimetre, foot, inch, metre, millimetre, yard
Height: centimetre, foot, inch, metre, millimetre, yard
Item Weight: gram, kilogram, microgram, milligram, ounce, pound, ton
Maximum Weight Recommendation: gram, kilogram, microgram, milligram, ounce, pound, ton
Voltage: kilovolt, millivolt, volt
Wattage: kilowatt, watt
Item Volume**: centilitre, cubic foot, cubic inch, cup, decilitre, fluid ounce, gallon, imperial gallon, litre, microlitre, millilitre, pint, quart

Ensure that each input is correctly formatted and limited to the allowed units for each entity. Any deviation from these units will be considered invalid.

### Instruction:
Extract the {entity_name} from the text

### Input:
{extracted_text}

### Response:
"""

    # Tokenize the input for the model
    inputs = tokenizer(
        [prompt_text],
        return_tensors="pt"
    ).to("cuda")  # Assuming the model is on CUDA

    # Generate the response
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the output
    decoded_output = tokenizer.batch_decode(outputs)[0]

    # Extract the response part after the ### Response: prompt
    response_start = "### Response:\n"
    response = decoded_output.split(response_start)[-1].strip("<eos>").strip()

    # Append the response to the list
    generated_responses.append(response)

# Store the responses in a DataFrame
response_df = pd.DataFrame({
      'prediction': generated_responses,
})



181it [04:06,  1.37s/it]

In [ ]:
response_df.to_csv('inf_centi_0-5k.csv', index = False)